# NOAA Data Extraction Method

This notebook shows an ongoing data extraction approach from the NOAA database which gives daily data for its 12361 weather stations globally. This data contains the station name, date, coordiantes, elevation, and various temperature attributes.

### Importing relevant modules

In [2]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt

### Testing Dataframe Structure
Below we see how the csv file from NOAA will be read into a dataframe for a given weather station. This data will be directly taken from the website. For 2019 for example, we pull the data from 
<a href="https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/2019">2019 Average Temperature Data</a>

In [3]:
sample_df = pd.read_csv('https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/2019/01001099999.csv')
sample_df

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,TEMP_ATTRIBUTES,DEWP,DEWP_ATTRIBUTES,...,MXSPD,GUST,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT
0,1001099999,2019-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",24.0,24,15.5,24,...,18.5,31.5,30.2,*,17.4,,0.00,G,999.9,1000
1,1001099999,2019-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",36.2,24,31.3,24,...,33.0,43.5,44.1,,28.2,,0.00,G,999.9,10000
2,1001099999,2019-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.5,24,31.6,24,...,29.9,46.4,39.2,*,33.6,,0.00,G,999.9,10000
3,1001099999,2019-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,24,32.5,24,...,22.1,42.7,41.0,,33.6,,0.29,G,999.9,100000
4,1001099999,2019-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,23,34.6,23,...,27.6,39.8,38.1,*,34.2,*,0.00,G,999.9,101000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,1001099999,2019-12-27,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",32.5,24,29.7,24,...,25.3,30.9,38.3,,19.6,,0.01,G,999.9,11000
361,1001099999,2019-12-28,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",33.6,24,30.8,24,...,35.0,44.5,39.6,*,29.5,*,0.77,G,999.9,11000
362,1001099999,2019-12-29,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",27.7,24,21.2,24,...,19.0,27.4,29.7,*,25.0,*,0.29,G,999.9,0
363,1001099999,2019-12-30,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",26.2,24,18.5,24,...,15.5,21.8,28.9,,24.4,,0.00,G,999.9,0


From this sample DataFrame we can see all the information given for a certain weather station. Those of interest to us will include the average temperature and highs and lows of that day.

Furthermore, the NOAA website has each weather station saved as a csv. A list of these csv names (named after the station id) is shown below:

In [4]:
weather_csv = pd.read_csv('weather_station_list.csv')
weather_csv

,file_name
0,01001099999.csv
1,01001499999.csv
2,01002099999.csv
3,01003099999.csv
4,01006099999.csv
...,...
12356,99999996405.csv
12357,99999996406.csv
12358,99999996407.csv
12359,99999996408.csv


We then want to convert this to a list:

In [7]:
weather_list = weather_csv['file_name'].to_list()
#showing the first few items from the list
weather_list[0:10]

['01001099999.csv',
 '01001499999.csv',
 '01002099999.csv',
 '01003099999.csv',
 '01006099999.csv',
 '01007099999.csv',
 '01008099999.csv',
 '01009099999.csv',
 '01010099999.csv',
 '01011099999.csv']

To first develop our code we will simply attempt creating a dataframe from the first three stations:

In [76]:
short_list = weather_list[0:3]
short_list

['01001099999.csv', '01001499999.csv', '01002099999.csv']

In [80]:
#creating an empty dataframe
full_data_short = pd.DataFrame()
#looping through each station of a list of stations
for station in short_list:
    #creating the correct URL and saving that csv into a temporary dataframe
    station_url = 'https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/2019/' + station
    temp_df = pd.read_csv(station_url)
    
    #taking that dataframe and saving only the aspects we want
    df2 = temp_df[['DATE','TEMP']]
    #column name will be the station ID with the data being the row name
    col = temp_df['STATION'][1]
    df2.rename(columns={"TEMP": str(col)}, inplace=True)
    df2.rename(columns={"DATE":"Station ID"}, inplace=True)
    #the date will be used as the index
    df2.set_index('Station ID', inplace = True)

    #the dataframe is then transposed so that we can add columns such as name, longitude and latitude
    df3 = df2.transpose()
    #taking parameters of interest from the original station dataframe
    lat = temp_df['LATITUDE'][1]
    lon = temp_df['LONGITUDE'][1]
    name = temp_df['NAME'][1]
    #adding these parameters to the dataframe
    df3.insert(0, "Latitude", lat, True)
    df3.insert(1, "Longitude", lon, True)
    df3.insert(2, "Name", name, True)
    
    #appending a stations data to the full dataframe
    full_data_short = full_data_short.append(df3, sort=False)

full_data_short

Station ID,Latitude,Longitude,Name,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,...,2019-12-22,2019-12-23,2019-12-24,2019-12-25,2019-12-26,2019-12-27,2019-12-28,2019-12-29,2019-12-30,2019-12-31
1001099999,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO",24.0,36.2,35.5,35.8,35.8,28.1,23.5,...,26.1,24.6,27.7,34.1,26.9,32.5,33.6,27.7,26.2,27.3
1001499999,59.791925,5.340850,"SORSTOKKEN, NO",39.7,36.4,36.5,45.6,NaN,42.5,38.9,...,45.7,42.6,NaN,NaN,35.6,38.7,NaN,46.4,43.5,NaN
1002099999,80.050000,16.250000,"VERLEGENHUKEN, NO",-0.1,5.1,19.7,14.3,16.1,20.0,18.8,...,17.2,20.1,21.6,14.7,15.9,19.5,20.8,18.5,9.8,5.9


We can further increase this number as proof of concept. It has been shown that the first 500 weather stations in this database revolve around scandinavia. The 500 weatherstations are distributed in finland, norway and sweden. We first create the list of stations of interest:

In [81]:
scand_station_list = weather_list[0:500]

We can then extract the data for this subset of scandinavia using the same code as above:

In [82]:
#creating an empty dataframe
full_data_scand = pd.DataFrame()
#looping through each station of a list of stations
for station in scand_station_list:
    #creating the correct URL and saving that csv into a temporary dataframe
    station_url = 'https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/2019/' + station
    temp_df = pd.read_csv(station_url)
    
    #taking that dataframe and saving only the aspects we want
    df2 = temp_df[['DATE','TEMP']]
    #column name will be the station ID with the data being the row name
    col = temp_df['STATION'][1]
    df2.rename(columns={"TEMP": str(col)}, inplace=True)
    df2.rename(columns={"DATE":"Station ID"}, inplace=True)
    #the date will be used as the index
    df2.set_index('Station ID', inplace = True)

    #the dataframe is then transposed so that we can add columns such as name, longitude and latitude
    df3 = df2.transpose()
    #taking parameters of interest from the original station dataframe
    lat = temp_df['LATITUDE'][1]
    lon = temp_df['LONGITUDE'][1]
    name = temp_df['NAME'][1]
    #adding these parameters to the dataframe
    df3.insert(0, "Latitude", lat, True)
    df3.insert(1, "Longitude", lon, True)
    df3.insert(2, "Name", name, True)
    
    #appending a stations data to the full dataframe
    full_data_scand = full_data_scand.append(df3, sort=False)

full_data_scand

Station ID,Latitude,Longitude,Name,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,...,2019-12-22,2019-12-23,2019-12-24,2019-12-25,2019-12-26,2019-12-27,2019-12-28,2019-12-29,2019-12-30,2019-12-31
1001099999,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO",24.0,36.2,35.5,35.8,35.8,28.1,23.5,...,26.1,24.6,27.7,34.1,26.9,32.5,33.6,27.7,26.2,27.3
1001499999,59.791925,5.340850,"SORSTOKKEN, NO",39.7,36.4,36.5,45.6,NaN,42.5,38.9,...,45.7,42.6,NaN,NaN,35.6,38.7,NaN,46.4,43.5,NaN
1002099999,80.050000,16.250000,"VERLEGENHUKEN, NO",-0.1,5.1,19.7,14.3,16.1,20.0,18.8,...,17.2,20.1,21.6,14.7,15.9,19.5,20.8,18.5,9.8,5.9
1003099999,77.000000,15.500000,"HORNSUND, NO",3.9,11.6,27.8,23.6,27.9,29.0,25.6,...,20.0,22.7,23.9,18.6,15.7,25.3,24.0,20.1,12.2,6.1
1006099999,78.250000,22.816667,"EDGEOYA, NO",5.2,4.3,14.2,16.0,18.8,18.9,21.2,...,6.8,17.9,23.0,12.3,14.8,18.4,13.3,10.1,-5.7,-5.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2744099999,61.500000,23.766667,"TAMPERE SATAKUNNANKATU, FI",33.4,22.8,15.5,27.1,21.3,27.6,32.4,...,37.1,37.4,34.2,32.7,30.7,23.0,18.0,33.1,40.1,32.9
2745099999,67.166667,29.183333,"SALLA NARUSKA, FI",19.7,0.0,-8.4,16.9,16.1,20.1,15.5,...,24.7,28.3,28.5,27.5,21.7,14.0,9.7,28.8,28.5,4.9
2746099999,59.816667,22.916667,"HANKO TULLINIEMI, FI",37.4,26.8,21.3,34.3,24.0,30.6,36.3,...,41.6,40.6,37.8,35.9,32.9,27.9,26.3,39.7,42.2,37.9
2747099999,60.383333,22.100000,"TURKU RAJAKARI, FI",36.7,27.4,21.8,35.0,25.5,31.4,35.3,...,39.9,38.5,37.9,35.8,33.2,28.4,24.4,38.8,42.9,38.2


We can save this data as a csv to use in tableau:

In [83]:
full_data_scand.to_csv(r'Scand.csv', index = False)

With some manipulation in tableau we can see the following map for a given day:
<img src="sample_scand.png">

This map can also be seen on tableau public at <a href="https://public.tableau.com/profile/mitchell.kitt#!/vizhome/Scandinavia/Sheet1?publish=yes">Tableau Public Map</a>

## NEXT STEPS:
- Get Full Data Set for all 12000+ Stations 
    - Might need a way to process all these as 500 take a few minutes
- Work with tableau to get slide bar so you can slide across time
- Manipulate data:
    - How many days below certain temp
    - Minimum temp days
